<a href="https://colab.research.google.com/github/ksolosina/BCIT/blob/master/lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задание: 1) Выберите набор данных (датасет) для решения задачи классификации или регресии. 2) В случае необходимости проведите удаление или заполнение пропусков и кодирование категориальных признаков. 3) С использованием метода train_test_split разделите выборку на обучающую и тестовую. 4) Обучите две ансамблевые модели. Оцените качество моделей с помощью одной из подходящих для задачи метрик. Сравните качество полученных моделей. 5) Произведите для каждой модели подбор значений одного гиперпараметра. В зависимости от используемой библиотеки можно применять функцию GridSearchCV, использовать перебор параметров в цикле, или использовать другие методы. 6) Повторите пункт 4 для найденных оптимальных значений гиперпараметров. Сравните качество полученных моделей с качеством моделей, полученных в пункте 4.

In [0]:
import numpy as np #библиотека для работы с многомерными массивами данных и математическими операциями над ними
import pandas as pd #библиотека для анализа и обработки данных
from scipy import stats
from sklearn.datasets import load_iris #берём датасет
import matplotlib.pyplot as plt #простое рисование графиков
import seaborn as sns #удобные дефолтные настройки графиков из matpotlib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import *
from sklearn.metrics import *
from sklearn.linear_model import LinearRegression # для линейной модели
from sklearn.svm import SVR # для SVM модели
from sklearn.tree import DecisionTreeRegressor # для дерева решений

from sklearn.ensemble import BaggingRegressor, RandomForestRegressor  #встроенные функции

%matplotlib inline
#для сохранения в ноутбуке вывода моих графиков

In [23]:
df = load_iris()
df = pd.DataFrame(data = np.c_[df['data'], df['target']], columns = df['feature_names'] + ['target'])
df.head() # первые 5 строк 

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [0]:

df.loc[:, df.columns!='target'] = df.loc[:, df.columns!='target'].apply(lambda x: x/x.max(), axis=0)

In [0]:
x_train, x_test, y_train, y_test = train_test_split(df.loc[:, df.columns!='target'], 
                                                    df['target'], 
                                                    test_size= 0.33) # разделяем на тест и трен

In [0]:
def print_stat(model):
    print(mean_absolute_error(model.predict(x_test), y_test))
    print(r2_score(model.predict(x_test), y_test))
    print(median_absolute_error(model.predict(x_test), y_test))

In [27]:
base = LinearRegression() #базова модель - линейная регрессия
BR = BaggingRegressor(base_estimator = base)
BR.fit(x_train, y_train)

BaggingRegressor(base_estimator=LinearRegression(copy_X=True,
                                                 fit_intercept=True,
                                                 n_jobs=None, normalize=False),
                 bootstrap=True, bootstrap_features=False, max_features=1.0,
                 max_samples=1.0, n_estimators=10, n_jobs=None, oob_score=False,
                 random_state=None, verbose=0, warm_start=False)

In [28]:
print_stat(BR)


0.15746533387017986
0.9318932527944914
0.10855453127031328


In [29]:
RFR = RandomForestRegressor(max_depth=3, random_state=0, #выборку разделяем на несколько частей и каждую предсказываем отдельно, а затем усредняем. используем деревья модели
                             n_estimators=100)
RFR.fit(x_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=3,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [30]:
print_stat(RFR)

0.06484175626774932
0.9499004682005062
0.001810631024985021


In [31]:
base = LinearRegression()
BR = BaggingRegressor(base_estimator = base)
BR_GV = GridSearchCV(BR, {'n_jobs':range(1,10)}, cv=3).fit(x_train, y_train).best_estimator_ #ищем лучшие бегенги и получаем статистику 

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [32]:
print_stat(BR_GV)

0.15866682889526226
0.9322825515621025
0.10631229316566548


In [33]:
RFR = RandomForestRegressor(random_state=0,
                             n_estimators=100)
RFR_GV = GridSearchCV(RFR, {'max_depth':range(1,10)}, cv=3).fit(x_train, y_train).best_estimator_ #ищем лучшие рендемфоресты

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [34]:
print_stat(RFR_GV)

0.07743188328566532
0.9543047319927822
0.032116026540843334
